# Quantum Crypto Key Exchange

In [6]:
#################################################
%matplotlib inline

# Imports of the Qiskit basic functionalities
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector, random_statevector, partial_trace
from qiskit.visualization import plot_distribution, plot_bloch_multivector
import random

def create_bell_pair():
    qc = QuantumCircuit(2, 2)
    qc.h(0)  # Hadamard gate on qubit 0
    qc.cx(0, 1)  # CNOT gate with control qubit 0 and target qubit 1
    return qc

def measure_in_random_basis(qc, qubit, classical_bit, basis):
    if basis == 'X':
        qc.h(qubit)
    qc.measure(qubit, classical_bit)
    
def bbm92_protocol(num_bits):
    alice_basis_choices = [random.choice(['X', 'Z']) for _ in range(num_bits)]
    bob_basis_choices = [random.choice(['X', 'Z']) for _ in range(num_bits)]
    
    alice_bits = []
    bob_bits = []
    
    for alice_basis, bob_basis in zip(alice_basis_choices, bob_basis_choices):
        # Create Bell pair
        qc = create_bell_pair()
        
        # Measure Alice's qubit in her chosen basis
        measure_in_random_basis(qc, 0, 0, alice_basis)
        
        # Measure Bob's qubit in his chosen basis
        measure_in_random_basis(qc, 1, 1, bob_basis)
        
        # Simulate the circuit
        job = Sampler().run(qc, shots = 10)

        # Collect the results from the job
        result = job.result()
        # Obtain the counts from the results
        statistics = result.quasi_dists[0].binary_probabilities()

        max_key = max(statistics, key=statistics.get)
        
        alice_bits.append(int(max_key[0]))
        bob_bits.append(int(max_key[1]))
    
    return alice_bits, bob_bits, alice_basis_choices, bob_basis_choices

num_bits = 100  # Example number of bits to generate
alice_bits, bob_bits, alice_bases, bob_bases = bbm92_protocol(num_bits)

print("Alice's bits:", alice_bits)
print("Bob's bits:", bob_bits)
print("Alice's bases:", alice_bases)
print("Bob's bases:", bob_bases)

Alice's bits: [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
Bob's bits: [1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
Alice's bases: ['X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', '

In [7]:
def sift_keys(alice_bits, bob_bits, alice_bases, bob_bases):
    sifted_key_alice = []
    sifted_key_bob = []
    
    for ab, bb, a_bit, b_bit in zip(alice_bases, bob_bases, alice_bits, bob_bits):
        if ab == bb:
            sifted_key_alice.append(a_bit)
            sifted_key_bob.append(b_bit)
    
    return sifted_key_alice, sifted_key_bob

sifted_key_alice, sifted_key_bob = sift_keys(alice_bits, bob_bits, alice_bases, bob_bases)

print("Sifted key (Alice):", sifted_key_alice)
print("Sifted key (Bob):", sifted_key_bob)

Sifted key (Alice): [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Sifted key (Bob): [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
